**중고차 가격 예측 데이터**
===

Attention
===

데이터 설명 : 중고차 가격 예측 데이터 (종속변수 :G3)   
x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv   
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv   
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv   
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv   
데이터 출처 :https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?select=vw.csv (참고, 데이터 수정)

In [2]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")


display(x_train.head())
display(y_train.head())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [3]:
x_train.nunique()

carID           4960
brand              9
model             90
year              24
transmission       4
mileage         3900
fuelType           5
tax               41
mpg              125
engineSize        37
dtype: int64

In [4]:
x_test.nunique()

carID           2672
brand              9
model             89
year              23
transmission       4
mileage         2226
fuelType           5
tax               34
mpg              107
engineSize        32
dtype: int64

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

drop_col = ['carID']

x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)
y_train_target = y_train['price']

# train에만 있고 test에는 없는 model들이 있기에 합쳐서 원핫 인코딩을 진행
# data leakage 이슈가 있으나 빅분기 시험에서는 이러한 방식으로 해결하는 것으로 태클 걸지 않음
# 정석은 train과 동일한 컬럼을 만들고 test에서 없는 경우에는 0으로 채우고, test에는 있으나 train에 없는 경우에는 데이터 제거

combined = pd.concat([x_train_drop, x_test_drop])
combined_encoded = pd.get_dummies(combined)

train_encoded = combined_encoded[:len(x_train_drop)]
test_encoded = combined_encoded[len(x_test_drop):]

xtr, xt, ytr, yt = train_test_split(train_encoded, y_train_target)

rf = RandomForestRegressor(random_state=42)
rf.fit(xtr, ytr)

y_validation_pred = rf.predict(xt)

y_validation_pred

array([19793.49, 42690.17, 19594.24, ..., 20803.85,  6141.31,  9584.97])

In [7]:
# 모델평가 
# mse : mean_squared_error / mae : mean_absolute_error  / mape : mean_absolute_percentage_error
# rmse : root_mean_squerd_error -> 패키지 없음 np.sqrt(mean_squared_error) 해줘야함

# mse
print('validation mse : ', mean_squared_error(yt, y_validation_pred))

# mae
print('validation mae : ', mean_absolute_error(yt, y_validation_pred))

# mape
print('validation mape : ', mean_absolute_percentage_error(yt, y_validation_pred))

# rmse
print('validation rmse : ', np.sqrt(mean_absolute_percentage_error(yt, y_validation_pred)))

# r2
print('validation r2 score : ', r2_score(yt, y_validation_pred))

validation mse :  16056714.79807634
validation mae :  2051.4428703368762
validation mape :  0.09869191221748791
validation rmse :  0.31415268933671076
validation r2 score :  0.9414509529248328
